# Fine-tuning SmolVLM on ChartLlama Dataset

This notebook demonstrates how to fine-tune the SmolVLM model on the ChartLlama dataset using parameter-efficient fine-tuning (LoRA).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
# Change directory to your code folder
os.chdir('/content/drive/MyDrive/code')

In [3]:
import torch
import gc
import os
import time
import numpy as np
from PIL import Image
from transformers import Idefics3Processor, Idefics3ForConditionalGeneration, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import random_split, Dataset
from chartllama_load import ChartDataset
import wandb
# Reset the framework state and reinitialize accelerator
from accelerate.utils import set_seed
from accelerate import Accelerator

In [4]:
!pip install -U bitsandbytes

In [5]:
# 1. Setup Configuration
WANDB_PROJECT = "smolvlm-chartllama"  # Set to None to disable wandb logging
MODEL_ID = "HuggingFaceTB/SmolVLM-Base"
OUTPUT_DIR = "./smolvlm-chartllama-improved"
DATA_DIR = "./chartllama_data"  # Update path as needed
SAMPLE_LIMIT = 300  # Use more samples for better performance
IMAGE_SIZE = 384
SEED = 42

In [6]:
# https://wandb.ai/authorize
# 0469802d14d997b8dad4d23a7ba212e0a8d8f197

In [7]:
# Configure wandb
if WANDB_PROJECT:
    wandb.init(project=WANDB_PROJECT)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ryan-seet467 (ryan-seet467-georgia-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Check Hardware and Clear Memory

In [8]:
# Check CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU: NVIDIA A100-SXM4-40GB
GPU memory: 42.47 GB


In [9]:
# Clear memory
torch.cuda.empty_cache()
gc.collect()

# Set environment variables for better memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Load Model and Processor

In [11]:
# Define model and paths
model_id = "HuggingFaceTB/SmolVLM-Base"
output_dir = "./smolvlm-chartllama-tuned"
data_dir = "./chartllama_data"  # Update this path accordingly

# Create output directory
os.makedirs(output_dir, exist_ok=True)

In [12]:
# 3. Load Processor and Model
print("Loading processor...")
processor = Idefics3Processor.from_pretrained(MODEL_ID)

Loading processor...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
print("Configuring model quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.float16
)

Configuring model quantization...


In [14]:
# Load model with quantization
print("Loading model...")
start_time = time.time()

model = Idefics3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
)

print(f"Model loaded in {time.time() - start_time:.2f} seconds")

Loading model...
Model loaded in 4.14 seconds


## Prepare Model for Fine-tuning

In [15]:
# Prepare model for k-bit training
print("Preparing model for training...")
model = prepare_model_for_kbit_training(model)

Preparing model for training...


In [16]:
# Configure LoRA
peft_config = LoraConfig(
    r=8,                      # Reduced from 16 to prevent overfitting
    lora_alpha=16,            # Scaled with rank
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.2,         # Increased dropout for better regularization
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head"]  # Save language modeling head for better output quality
)

In [17]:
# Apply LoRA to model
model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

# Disable caching for training (saves memory)
model.config.use_cache = False

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 105,308,160 || all params: 2,351,581,040 || trainable%: 4.4782


## Load Dataset

In [18]:
# Load ChartLlama dataset
print("Loading dataset...")
dataset = ChartDataset(
    data_dir=DATA_DIR,
    processor=processor,
    image_size=384,  # Use same size throughout
)

Loading dataset...


Loading JSON files: 100%|██████████| 7/7 [00:14<00:00,  2.07s/it]


In [19]:
# Split dataset into train, validation, and test
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset, remaining = random_split(
    dataset, [train_size, val_size + test_size],
    generator=torch.Generator().manual_seed(SEED)
)

val_dataset, test_dataset = random_split(
    remaining, [val_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"Training on {len(train_dataset)} samples, validating on {len(val_dataset)} samples, testing on {len(test_dataset)} samples")


Training on 784 samples, validating on 98 samples, testing on 98 samples


In [20]:
def collate_fn(examples):
    batch = {}

    # Process text inputs
    if 'input_ids' in examples[0]:
        # Get max length
        max_length = max(example['input_ids'].size(0) for example in examples)

        # Create padded batches
        input_ids = []
        attention_mask = []
        labels = []

        for example in examples:
            # Get current lengths
            curr_len = example['input_ids'].size(0)
            pad_len = max_length - curr_len

            # Pad input_ids (with 0) and attention_mask (with 0)
            input_ids.append(torch.cat([
                example['input_ids'],
                torch.zeros(pad_len, dtype=example['input_ids'].dtype)
            ]))

            attention_mask.append(torch.cat([
                example['attention_mask'],
                torch.zeros(pad_len, dtype=example['attention_mask'].dtype)
            ]))

            # Pad labels with -100 (ignored in loss calculation)
            labels.append(torch.cat([
                example['labels'],
                torch.full((pad_len,), -100, dtype=example['labels'].dtype)
            ]))

        batch['input_ids'] = torch.stack(input_ids)
        batch['attention_mask'] = torch.stack(attention_mask)
        batch['labels'] = torch.stack(labels)

    return batch

## Configure Training

In [21]:
# Enhanced training settings
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,  # Effective batch size of 16
    max_steps=50,                  # Train longer
    learning_rate=5e-6,             # Lower learning rate
    fp16=True,
    logging_steps=50,
    output_dir="./smolvlm-chartllama-improved",
    optim="adamw_torch",
    weight_decay=0.05,              # Strong weight decay
    warmup_ratio=0.1,               # Gradual warmup
    lr_scheduler_type="cosine",     # Cosine decay
    save_strategy="steps",
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,    # Use early stopping
    metric_for_best_model="eval_loss"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# 7. Initialize and Run Trainer
print("Initializing trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Initializing trainer...


In [23]:
# Create a new accelerator
accelerator = Accelerator()

# Set seed for reproducibility
set_seed(SEED)

## Training

In [24]:
# Start training
print("Starting training...")
start_time = time.time()

trainer.train()

training_time = (time.time() - start_time) / 60
print(f"Training completed in {training_time:.2f} minutes")

# 8. Save the Model
print("Saving fine-tuned model...")
trainer.save_model(OUTPUT_DIR)

# Also save the processor
processor.save_pretrained(OUTPUT_DIR)


Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Training completed in 10.65 minutes
Saving fine-tuned model...


['./smolvlm-chartllama-improved/processor_config.json']

## Test Inference

In [25]:
from transformers import Idefics3Processor, Idefics3ForConditionalGeneration
from transformers import BitsAndBytesConfig
from peft import PeftConfig, PeftModel
import torch
import time
from PIL import Image

def load_models(base_model_id="HuggingFaceTB/SmolVLM-Base",
                lora_path=None,
                quantize_base=False,
                quantize_finetuned=False):
    """
    Load base and fine-tuned models with the same quantization settings.

    Args:
        base_model_id: HF model ID for the base model
        lora_path: Path to fine-tuned LoRA weights
        quantize_base: Whether to quantize the base model
        quantize_finetuned: Whether to quantize the fine-tuned model

    Returns:
        tuple: (processor, base_model, fine_tuned_model)
    """
    # Load processor (shared between models)
    processor = Idefics3Processor.from_pretrained(base_model_id)

    # Configure quantization if requested
    base_kwargs = {"device_map": "auto"}
    if quantize_base:
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_quant_type="nf8",
            bnb_8bit_compute_dtype=torch.float16
        )
        base_kwargs["quantization_config"] = bnb_config
    else:
        base_kwargs["torch_dtype"] = torch.float16

    # Load base model
    print("Loading base model...")
    base_model = Idefics3ForConditionalGeneration.from_pretrained(
        base_model_id,
        **base_kwargs
    )

    # Load fine-tuned model if path provided
    fine_tuned_model = None
    if lora_path:
        print(f"Loading fine-tuned model from {lora_path}...")
        fine_tuned_model = PeftModel.from_pretrained(
            base_model,
            lora_path
        )

    return processor, base_model, fine_tuned_model

def run_inference(model, processor, image, question, max_length=128, max_new_tokens=64):
    """
    Run inference with either base or fine-tuned model.

    Args:
        model: Base or fine-tuned model
        processor: Tokenizer/processor for the model
        image: PIL image or path to image
        question: Text prompt/question
        max_length: Maximum sequence length
        max_new_tokens: Maximum new tokens to generate

    Returns:
        str: Generated response
    """
    # Ensure image is a PIL Image
    if isinstance(image, str):
        image = Image.open(image).convert('RGB')

    # Add <image> token if not present
    if "<image>" not in question:
        question += " <image>"

    # Prepare inputs
    inputs = processor(
        text=question,
        images=image,
        return_tensors="pt"
    )

    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Start timing
    start_time = time.time()

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False  # Use greedy decoding for consistency
        )

    # Calculate elapsed time
    elapsed = time.time() - start_time

    # Decode the generated tokens
    generated_text = processor.decode(outputs[0], skip_special_tokens=True)

    # Extract the response (remove the question)
    response = generated_text.split("<image>")[-1].strip()

    return {
        "response": response,
        "time_seconds": elapsed
    }

def compare_models(dataset, sample_indices=None,
                  base_model_id="HuggingFaceTB/SmolVLM-Instruct",
                  lora_path="./smolvlm-chartllama-improved",
                  quantize=True):
    """
    Compare base and fine-tuned models on the same samples.

    Args:
        dataset: ChartDataset instance
        sample_indices: List of sample indices to test (defaults to [0, 100, 200])
        base_model_id: HF model ID for the base model
        lora_path: Path to fine-tuned LoRA weights
        quantize: Whether to use quantization for both models

    Returns:
        dict: Comparison results
    """
    # Default sample indices if none provided
    if sample_indices is None:
        sample_indices = [0, 100, 200] if len(dataset) >= 201 else [0]

    # Load models
    processor, base_model, fine_tuned_model = load_models(
        base_model_id=base_model_id,
        lora_path=lora_path,
        quantize_base=quantize,
        quantize_finetuned=quantize
    )

    # Store results
    results = {}

    # Test each sample
    for idx in sample_indices:
        if idx >= len(dataset):
            print(f"Sample index {idx} out of range. Dataset has {len(dataset)} samples.")
            continue

        # Get sample data
        sample = dataset.data[idx]
        image = sample['image']
        question = sample['text']
        expected = sample['answer']
        chart_type = sample.get('chart_type', 'unknown')

        print(f"\n--- Testing sample {idx} (chart type: {chart_type}) ---")
        print(f"Question: {question}")
        print(f"Expected: {expected}")

        # Base model inference
        base_result = run_inference(base_model, processor, image, question)
        print(f"\nBase model response ({base_result['time_seconds']:.2f}s):")
        print(base_result['response'])

        # Fine-tuned model inference
        if fine_tuned_model:
            ft_result = run_inference(fine_tuned_model, processor, image, question)
            print(f"\nFine-tuned model response ({ft_result['time_seconds']:.2f}s):")
            print(ft_result['response'])
        else:
            ft_result = None

        # Store results
        results[idx] = {
            "question": question,
            "expected": expected,
            "chart_type": chart_type,
            "base_model": base_result,
            "fine_tuned": ft_result
        }

    return results


In [26]:
# Example usage:
results = compare_models(dataset, sample_indices=[0, 10, 20])

Loading base model...
Loading fine-tuned model from ./smolvlm-chartllama-improved...

--- Testing sample 0 (chart type: heatmap) ---
Question: <image>
What was the number of road safety incidents in South America in 2018?
Expected: 7000

Base model response (7.61s):
What was the number of road safety incidents in South America in 2018? 7000.0.
What was the number of road safety incidents in Europe in 2018? 7000.0.
What was the number of road safety incidents in Asia in 2018? 4500.0.
What was the

Fine-tuned model response (7.98s):
What was the number of road safety incidents in South America in 2018? 7000.0.
What was the number of road safety incidents in Europe in 2018? 7000.0.
What was the number of road safety incidents in Asia in 2018? 4500.0.
What was the

--- Testing sample 10 (chart type: heatmap) ---
Question: <image>
What is the range of years represented in the chart?
Expected: 2000 to 2007

Base model response (7.71s):
What is the range of years represented in the chart?
[20